# Pyspark Handling Missing Values
- dropping columns 
- dropping rows
- various parameter in dropping functionalities
- handling missing values by mean

In [189]:
import pandas as pd
import numpy as np

df = pd.DataFrame([["Tom", 31, 10,3000],["Daniel",30,8,4000],["Ron",29,4,5000],['',34,10,],['',36],["Fredric",'','',38000],[''],["Snape",40,10,'']], columns = (["Name","Age","Experience","Salary"]))
df.replace('',np.nan, inplace = True)
df.to_csv("test3.csv", index = False)

df

,Name,Age,Experience,Salary
0,Tom,31.0,10.0,3000.0
1,Daniel,30.0,8.0,4000.0
2,Ron,29.0,4.0,5000.0
3,NaN,34.0,10.0,NaN
4,NaN,36.0,NaN,NaN
5,Fredric,NaN,NaN,38000.0
6,NaN,NaN,NaN,NaN
7,Snape,40.0,10.0,NaN


In [190]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [191]:
df_pyspark = spark.read.csv("test3.csv",header = True, inferSchema=True)
df_pyspark.show()

+-------+----+----------+-------+
|   Name| Age|Experience| Salary|
+-------+----+----------+-------+
|    Tom|31.0|      10.0| 3000.0|
| Daniel|30.0|       8.0| 4000.0|
|    Ron|29.0|       4.0| 5000.0|
|   null|34.0|      10.0|   null|
|   null|36.0|      null|   null|
|Fredric|null|      null|38000.0|
|   null|null|      null|   null|
|  Snape|40.0|      10.0|   null|
+-------+----+----------+-------+



### Dropping columns

In [192]:
df_pyspark.drop("Salary").show()

+-------+----+----------+
|   Name| Age|Experience|
+-------+----+----------+
|    Tom|31.0|      10.0|
| Daniel|30.0|       8.0|
|    Ron|29.0|       4.0|
|   null|34.0|      10.0|
|   null|36.0|      null|
|Fredric|null|      null|
|   null|null|      null|
|  Snape|40.0|      10.0|
+-------+----+----------+



### Dropping null Values

In [193]:
# all rows having null values are removed
df_pyspark.na.drop().show()

+------+----+----------+------+
|  Name| Age|Experience|Salary|
+------+----+----------+------+
|   Tom|31.0|      10.0|3000.0|
|Daniel|30.0|       8.0|4000.0|
|   Ron|29.0|       4.0|5000.0|
+------+----+----------+------+



In [194]:
# drop only if all values are null in a row
df_pyspark.na.drop(how = "all").show()

+-------+----+----------+-------+
|   Name| Age|Experience| Salary|
+-------+----+----------+-------+
|    Tom|31.0|      10.0| 3000.0|
| Daniel|30.0|       8.0| 4000.0|
|    Ron|29.0|       4.0| 5000.0|
|   null|34.0|      10.0|   null|
|   null|36.0|      null|   null|
|Fredric|null|      null|38000.0|
|  Snape|40.0|      10.0|   null|
+-------+----+----------+-------+



In [195]:
# adding threshold for dropping nan values

df_pyspark.na.drop(how = "all", thresh=2).show()

+-------+----+----------+-------+
|   Name| Age|Experience| Salary|
+-------+----+----------+-------+
|    Tom|31.0|      10.0| 3000.0|
| Daniel|30.0|       8.0| 4000.0|
|    Ron|29.0|       4.0| 5000.0|
|   null|34.0|      10.0|   null|
|Fredric|null|      null|38000.0|
|  Snape|40.0|      10.0|   null|
+-------+----+----------+-------+



row containing more than 2 null values have been dropped in the table above

In [196]:
## subset feature for dropping null values from a specific column
df_pyspark.na.drop(how = "all", subset = ["Experience"]).show()

+------+----+----------+------+
|  Name| Age|Experience|Salary|
+------+----+----------+------+
|   Tom|31.0|      10.0|3000.0|
|Daniel|30.0|       8.0|4000.0|
|   Ron|29.0|       4.0|5000.0|
|  null|34.0|      10.0|  null|
| Snape|40.0|      10.0|  null|
+------+----+----------+------+



in the table above any rows containing null values for experience have been removed

## Filling the Missing Values
* string column can only be replaced with strings and numeric columns can only be replaced by numbers

In [197]:

df_pyspark.na.fill("").show()

+-------+----+----------+-------+
|   Name| Age|Experience| Salary|
+-------+----+----------+-------+
|    Tom|31.0|      10.0| 3000.0|
| Daniel|30.0|       8.0| 4000.0|
|    Ron|29.0|       4.0| 5000.0|
|       |34.0|      10.0|   null|
|       |36.0|      null|   null|
|Fredric|null|      null|38000.0|
|       |null|      null|   null|
|  Snape|40.0|      10.0|   null|
+-------+----+----------+-------+



In [198]:
df_pyspark.na.fill(0).show()

+-------+----+----------+-------+
|   Name| Age|Experience| Salary|
+-------+----+----------+-------+
|    Tom|31.0|      10.0| 3000.0|
| Daniel|30.0|       8.0| 4000.0|
|    Ron|29.0|       4.0| 5000.0|
|   null|34.0|      10.0|    0.0|
|   null|36.0|       0.0|    0.0|
|Fredric| 0.0|       0.0|38000.0|
|   null| 0.0|       0.0|    0.0|
|  Snape|40.0|      10.0|    0.0|
+-------+----+----------+-------+



In [199]:
# Handling for a specific columns, 
df_pyspark.na.fill({'Salary':0}).show()

+-------+----+----------+-------+
|   Name| Age|Experience| Salary|
+-------+----+----------+-------+
|    Tom|31.0|      10.0| 3000.0|
| Daniel|30.0|       8.0| 4000.0|
|    Ron|29.0|       4.0| 5000.0|
|   null|34.0|      10.0|    0.0|
|   null|36.0|      null|    0.0|
|Fredric|null|      null|38000.0|
|   null|null|      null|    0.0|
|  Snape|40.0|      10.0|    0.0|
+-------+----+----------+-------+



#### Replacing null values with mean

In [201]:
# drop  if all values are null in a row
df_pyspark = df_pyspark.na.drop(how = "all")


# replace null values using mean
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols = ['Age','Experience','Salary'],
                  outputCols = ["{}_imputed".format(c) for c in ['Age','Experience','Salary']]).setStrategy("mean")


# Add imputation cols to df_pyspark

imputer.fit(df_pyspark).transform(df_pyspark).show()



+-------+----+----------+-------+------------------+------------------+--------------+
|   Name| Age|Experience| Salary|       Age_imputed|Experience_imputed|Salary_imputed|
+-------+----+----------+-------+------------------+------------------+--------------+
|    Tom|31.0|      10.0| 3000.0|              31.0|              10.0|        3000.0|
| Daniel|30.0|       8.0| 4000.0|              30.0|               8.0|        4000.0|
|    Ron|29.0|       4.0| 5000.0|              29.0|               4.0|        5000.0|
|   null|34.0|      10.0|   null|              34.0|              10.0|       12500.0|
|   null|36.0|      null|   null|              36.0|               8.4|       12500.0|
|Fredric|null|      null|38000.0|33.333333333333336|               8.4|       38000.0|
|  Snape|40.0|      10.0|   null|              40.0|              10.0|       12500.0|
+-------+----+----------+-------+------------------+------------------+--------------+



#### Replacing null values with median

In [202]:


# replace null values using median
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols = ['Age','Experience','Salary'],
                  outputCols = ["{}_imputed".format(c) for c in ['Age','Experience','Salary']]).setStrategy("median")


# Add imputation cols to df_pyspark

imputer.fit(df_pyspark).transform(df_pyspark).show()

+-------+----+----------+-------+-----------+------------------+--------------+
|   Name| Age|Experience| Salary|Age_imputed|Experience_imputed|Salary_imputed|
+-------+----+----------+-------+-----------+------------------+--------------+
|    Tom|31.0|      10.0| 3000.0|       31.0|              10.0|        3000.0|
| Daniel|30.0|       8.0| 4000.0|       30.0|               8.0|        4000.0|
|    Ron|29.0|       4.0| 5000.0|       29.0|               4.0|        5000.0|
|   null|34.0|      10.0|   null|       34.0|              10.0|        4000.0|
|   null|36.0|      null|   null|       36.0|              10.0|        4000.0|
|Fredric|null|      null|38000.0|       31.0|              10.0|       38000.0|
|  Snape|40.0|      10.0|   null|       40.0|              10.0|        4000.0|
+-------+----+----------+-------+-----------+------------------+--------------+

